In [53]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager


In [54]:
url = 'https://mars.nasa.gov/news'

In [55]:
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [56]:
title_results = soup.find('div', class_="content_title")
news_title = title_results.get_text(strip=True)


In [57]:
para_results = soup.find('div', class_="rollover_description_inner")
news_p = para_results.get_text(strip=True)

In [58]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/jeremykalmus/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache
 


In [59]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [60]:
footer = soup.find('footer')
for a  in footer.find_all('a'):
    featured_image = a['data-fancybox-href']
    featured_image_url = f'https://www.jpl.nasa.gov/{featured_image}'

'https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA17896_ip.jpg'

In [84]:
url = 'https://space-facts.com/mars/'
mars_data_table = pd.read_html(url, header=0)
mars_data_table = mars_data_table[0]
mars_data_html = mars_data_table.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Equatorial Diameter:</th>\n      <th>6,792 km</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>Recorded By:</td>\n      <td>Eg

In [83]:
hemi_list = ['Cerberus', 'Schiaparelli', 'Syrtis_Major', 'Valles_Marineris']
hemi = 'Cerberus'
hemisphere_image_urls = []

for hemi in hemi_list:
    hemi_url = f'https://astrogeology.usgs.gov/search/map/Mars/Viking/{hemi}_enhanced'
    response = requests.get(hemi_url)
    soup = bs(response.text, 'html.parser')

    downloads = soup.find('div', class_="downloads" )
    images_list = downloads.find_all('li')[1]
    for a in images_list.find_all('a'):
        img_url = a['href']
        title = f'{hemi} Hemisphere'
        title = title.replace('_', ' ')
        image_dict = {'title': title, 'img_url':img_url}
    hemisphere_image_urls.append(image_dict)    

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]

In [86]:
mars_dict = ({'Recent News':news_title, 'News Story':news_p }, {'Featured Image':featured_image_url}, {'Mars Data': mars_data_html}, {'Mars Hemisphers': hemisphere_image_urls} )